In [1]:
import sys
import json
import requests
from IPython.display import Audio

# AivisSpeech

In [2]:
def create_voice_wav(text):
  endpoint = "http://localhost:8080/api-aivisspeech"

  # get style id
  endpoint_speakers = endpoint + "/speakers"
  res = requests.get(endpoint_speakers)
  speakers = json.loads(res.content)
  style_id = speakers[0]["styles"][3]["id"]

  # get audio query
  endpoint_audioquery = endpoint + "/audio_query"
  params = {"speaker": style_id, "text": text}
  res = requests.post(endpoint_audioquery, params=params)
  audioquery_json = json.loads(res.content)

  # create audio data
  endpoint_synthesis = endpoint + "/synthesis"
  params = {"speaker": style_id}
  res = requests.post(endpoint_synthesis, params=params, json=audioquery_json)

  return res.content

In [3]:
waiting_audio = create_voice_wav("ご質問ありがとうございます！少々お待ちください，ちょっと考えますね．うーんと．")
Audio(waiting_audio)

# Gemma2

In [4]:
def create_text_response(prompt):
  endpoint = "http://localhost:8080/api-gemma2/chat"

  message_json = {"message": prompt}
  res = requests.post(endpoint, json=message_json, stream=True)

  total_sentence = ""
  for chunk in res.iter_content(chunk_size=50, decode_unicode=True):
      if chunk:
          sys.stdout.write(chunk)
          sys.stdout.flush()
          total_sentence += chunk

  return total_sentence

In [10]:
message = """あなたは先生です．以下のことに注意して回答してください．
* 箇条書きを避ける
* 丁寧な言葉遣いを心がける
* 自然な会話文で回答する
* 特殊文字の使用を避ける
質問: """
message += "LLMと合成音声の組み合わせについてどう思いますか．"

_ = create_text_response(message)

# AivisSpeech * Gemma2

In [6]:
def clean_text(text):
  text = text.replace(" *", "")
  text = text.replace("* ", "")
  text = text.replace("*", "")
  return text

In [7]:
def create_voice_response(prompt, waiting_audio=waiting_audio):
  display(Audio(waiting_audio))

  endpoint = "http://localhost:8080/api-gemma2/chat"
  message_json = {"message": prompt}
  res = requests.post(endpoint, json=message_json, stream=True)

  current_chunk = ""
  for chunk in res.iter_content(chunk_size=50, decode_unicode=True):
      if chunk:
          current_chunk += chunk
          if "。" in current_chunk:
            sentences = current_chunk.split("。")[:-1]
            current_chunk = current_chunk.split("。")[-1]
            for sentence in sentences:
              print(sentence)
              audio = create_voice_wav(clean_text(sentence))
              display(Audio(audio))

  return 

In [8]:
message = """あなたは先生です．以下のことに注意して回答してください．
* 箇条書きを避ける
* 丁寧な言葉遣いを心がける
* 自然な会話文で回答する
* 特殊文字の使用を避ける
質問: """
message += "LLMと合成音声の組み合わせについてどう思いますか．"

create_voice_response(message)


LLMと合成音声の組み合わせは、非常に興味深い技術です。 

まず、LLMの能力は、文章生成、翻訳、要約など、様々なタスクをこなすことができます。一方で、合成音声は、自然で人間らしい声で、テキストを伝えるための重要な要素です。 

これらの技術を組み合わせることで、より人間らしい会話や、複雑な情報をより分かりやすく伝えることができます。例えば、顧客からの問い合わせに対して、LLMが回答を生成し、合成音声でそれを伝えることで、よりスムーズで効率的なサービスを実現できます。 



 
